# Working with SQL Databases

sqlite3:

The sqlite3 module allows interaction with SQLite databases, which are self-contained, serverless, and zero-configuration.



In [ ]:
# main.py
import sqlite3

# Connect to SQLite database
conn = sqlite3.connect('vehicles.db')
cursor = conn.cursor()

# Create table if not exists
cursor.execute('''
CREATE TABLE IF NOT EXISTS vehicles (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    make TEXT,
    model TEXT,
    year INTEGER,
    price REAL
)
''')

# Add vehicle to database
def add_vehicle_to_db(vehicle):
    cursor.execute('''
    INSERT INTO vehicles (make, model, year, price) VALUES (?, ?, ?, ?)
    ''', (vehicle.make, vehicle.model, vehicle.year, vehicle.price))
    conn.commit()

# Retrieve all vehicles
def get_all_vehicles():
    cursor.execute('SELECT * FROM vehicles')
    return cursor.fetchall()

# Example usage
vehicle = Vehicle(make="Toyota", model="Corolla", year=2020, price=20000)
add_vehicle_to_db(vehicle)
all_vehicles = get_all_vehicles()
print(all_vehicles)


SQLAlchemy:

SQLAlchemy is an SQL toolkit and Object-Relational Mapping (ORM) library for Python.

In [ ]:
# main.py
from sqlalchemy import create_engine, Column, Integer, String, Float
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker

# Database setup
engine = create_engine('sqlite:///vehicles.db')
Base = declarative_base()
Session = sessionmaker(bind=engine)
session = Session()

# Vehicle model
class Vehicle(Base):
    __tablename__ = 'vehicles'
    id = Column(Integer, primary_key=True)
    make = Column(String)
    model = Column(String)
    year = Column(Integer)
    price = Column(Float)

# Create tables
Base.metadata.create_all(engine)

# Add a vehicle
def add_vehicle(vehicle):
    session.add(vehicle)
    session.commit()

# Retrieve all vehicles
def get_all_vehicles():
    return session.query(Vehicle).all()

# Example usage
vehicle = Vehicle(make="Toyota", model="Corolla", year=2020, price=20000)
add_vehicle(vehicle)
all_vehicles = get_all_vehicles()
print(all_vehicles)


Django ORM:

Django ORM is part of the Django web framework and provides a powerful, high-level abstraction for database interactions.

In [ ]:
# models.py in your Django app
from django.db import models

class Vehicle(models.Model):
    make = models.CharField(max_length=100)
    model = models.CharField(max_length=100)
    year = models.IntegerField()
    price = models.FloatField()

# In a Django view or script
from .models import Vehicle

# Add a vehicle
vehicle = Vehicle(make="Toyota", model="Corolla", year=2020, price=20000)
vehicle.save()

# Retrieve all vehicles
all_vehicles = Vehicle.objects.all()
print(all_vehicles)


# NoSQL Databases

MongoDB:

MongoDB is a NoSQL database that uses JSON-like documents to store data.

In [ ]:
# main.py
from pymongo import MongoClient

# Connect to MongoDB
client = MongoClient('localhost', 27017)
db = client['vehicle_management']
vehicles_collection = db['vehicles']

# Add vehicle to MongoDB
def add_vehicle_to_mongo(vehicle):
    vehicle_data = {
        'make': vehicle.make,
        'model': vehicle.model,
        'year': vehicle.year,
        'price': vehicle.price
    }
    vehicles_collection.insert_one(vehicle_data)

# Retrieve all vehicles from MongoDB
def get_all_vehicles_from_mongo():
    return list(vehicles_collection.find())

# Example usage
vehicle = Vehicle(make="Toyota", model="Corolla", year=2020, price=20000)
add_vehicle_to_mongo(vehicle)
all_vehicles = get_all_vehicles_from_mongo()
print(all_vehicles)


Redis:

Redis is an in-memory data structure store used as a database, cache, and message broker.

In [ ]:
# main.py
import redis
import json

# Connect to Redis
r = redis.Redis(host='localhost', port=6379, db=0)

# Add vehicle to Redis
def add_vehicle_to_redis(vehicle):
    vehicle_data = {
        'make': vehicle.make,
        'model': vehicle.model,
        'year': vehicle.year,
        'price': vehicle.price
    }
    r.set(f"vehicle:{vehicle.id}", json.dumps(vehicle_data))

# Retrieve vehicle from Redis
def get_vehicle_from_redis(vehicle_id):
    vehicle_data = r.get(f"vehicle:{vehicle_id}")
    if vehicle_data:
        return json.loads(vehicle_data)
    return None

# Example usage
vehicle = Vehicle(make="Toyota", model="Corolla", year=2020, price=20000)
add_vehicle(vehicle)  # Ensure vehicle.id is set
add_vehicle_to_redis(vehicle)
retrieved_vehicle = get_vehicle_from_redis(vehicle.id)
print(retrieved_vehicle)


# Understanding Database Design and Optimization


1. Normalization: Process of organizing the columns and tables of a database to reduce data redundancy and improve data integrity.

2. Relationships: Define how data in one table is related to data in another table.

3. Indexes: Use indexes to speed up retrieval of data.

In [ ]:
import sqlite3

# Connect to the SQLite database (or create it if it doesn't exist)
conn = sqlite3.connect('vehicle_management.db')
cursor = conn.cursor()

#Create table for manufacturers
CREATE TABLE manufacturers (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    name TEXT NOT NULL
);

#Create table for owners
CREATE TABLE owners (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    name TEXT NOT NULL
);

#Create table for vehicles with foreign keys to manufacturers and owners
CREATE TABLE vehicles (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    make TEXT NOT NULL,
    model TEXT NOT NULL,
    year INTEGER NOT NULL,
    price REAL NOT NULL,
    manufacturer_id INTEGER,
    owner_id INTEGER,
    FOREIGN KEY (manufacturer_id) REFERENCES manufacturers(id),
    FOREIGN KEY (owner_id) REFERENCES owners(id)
);

# Create indexes for optimization
CREATE INDEX idx_vehicle_make ON vehicles(make);
CREATE INDEX idx_vehicle_model ON vehicles(model);
CREATE INDEX idx_vehicle_year ON vehicles(year);

# Function to add a manufacturer
def add_manufacturer(name):
    cursor.execute('INSERT INTO manufacturers (name) VALUES (?)', (name,))
    conn.commit()
    return cursor.lastrowid

# Function to add an owner
def add_owner(name):
    cursor.execute('INSERT INTO owners (name) VALUES (?)', (name,))
    conn.commit()
    return cursor.lastrowid

# Function to add a vehicle
def add_vehicle(make, model, year, price, manufacturer_id, owner_id):
    cursor.execute('''
    INSERT INTO vehicles (make, model, year, price, manufacturer_id, owner_id)
    VALUES (?, ?, ?, ?, ?, ?)
    ''', (make, model, year, price, manufacturer_id, owner_id))
    conn.commit()
    return cursor.lastrowid

# Function to retrieve vehicles by make
def get_vehicles_by_make(make):
    cursor.execute('SELECT * FROM vehicles WHERE make = ?', (make,))
    return cursor.fetchall()

manufacturer_id = add_manufacturer('Toyota')
owner_id = add_owner('John Doe')
vehicle_id = add_vehicle('Corolla', 'Sedan', 2020, 20000, manufacturer_id, owner_id)

vehicles = get_vehicles_by_make('Corolla')
print(vehicles)

# Close the database connection
conn.close()
